In [ ]:
import os
import random

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import StanfordCars, MNIST
from torchvision import transforms

from models.vgg import VGG16
from models.resnet import ResNet18

In [ ]:
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

In [ ]:
config = {
    'data_path': '../datasets',
    'batch_size': 256,
    'learning_rate': 1e-3,
    'epochs': 50,
    'test_epoch': 10,
    'num_classes': 196,
    'save_path': '../trained models/resnet18.pth'
}

### Dataset

In [ ]:
data_types = ['train', 'test']

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize([256, 256]),
        transforms.RandomCrop([224, 224]),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [ ]:
datasets = {data_type: StanfordCars(config['data_path'], data_type, transform=data_transforms[data_type], download=True) for data_type in data_types}
dataloaders = {data_type: DataLoader(datasets[data_type], config['batch_size'], shuffle=True if data_type is 'train' else False, num_workers=8, pin_memory=True) for data_type in data_types}
dataset_sizes = {data_type: len(datasets[data_type]) for data_type in data_types}
class_names = datasets['train'].classes

### Display Sample Image

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

In [ ]:
num_sample_images = 4
inputs, classes = next(iter(dataloaders['train']))
out = torchvision.utils.make_grid(inputs[:num_sample_images])
imshow(out, title=[class_names[x] for x in classes[:num_sample_images]])

### Train

In [ ]:
# model = torchvision.models.vgg11(pretrained=False)
# for param in model.parameters():
#     param.requires_grad = False
    
# model.classifier[6] = nn.Linear(in_features=4096, out_features=config['num_classes'])
# model = torchvision.models.resnet18(pretrained=False)
# model.fc = nn.Linear(model.fc.in_features, config['num_classes'])


In [ ]:
model = ResNet18(config['num_classes'])

In [ ]:
model = model.to(device)
print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), config['learning_rate'], momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.25)

In [ ]:
def train(model, criterion, optimizer, scheduler):
    for epoch in range(1, config['epochs'] + 1):
        print(f'epoch: {epoch}')
        
        phases = ['train']
        if epoch % config['test_epoch'] == 0:
            phases = ['train', 'test']
        
        for phase in phases:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            epoch_loss = 0.0
            epoch_acc = 0.0
            epoch_corrects = 0
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                epoch_loss += loss.item() * inputs.size(0)
                epoch_corrects += torch.sum(preds.data == labels.data)
            
            epoch_loss /= dataset_sizes[phase]
            epoch_acc = epoch_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
        scheduler.step()
        
    return model
            

In [ ]:
model = train(model, criterion, optimizer, scheduler)

In [ ]:
torch.save(model.state_dict(), config['save_path'])

In [ ]:
def test(model):
    model.eval()
            
    epoch_loss = 0.0
    epoch_acc = 0.0
    epoch_corrects = 0
    for inputs, labels, in tqdm(dataloaders['test']):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
        
        epoch_loss += loss.item() * inputs.size(0)
        epoch_corrects += torch.sum(preds.data == labels.data)
    
    epoch_loss /= dataset_sizes['test']
    epoch_acc = epoch_corrects.double() / dataset_sizes['test']
    
    print(f'Test Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            

In [ ]:
test(model)